In [148]:
#library

import pandas as pd
import contractions
import re
import string
from cleantext import clean
from nltk.tokenize import word_tokenize
import nltk
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.corpus import wordnet

In [149]:
#data cleansing

def cleansing(tweet):
    #remove RT Tag
    t1 = re.sub('RT\s', '', tweet)
    
    #remove @_username
    t2 = re.sub('\B@\w+', '', t1)
    
    #remove emoji
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    
    t3 = emoji_pattern.sub(r'', t2)
    
    #remove URL (https:// or https://)
    t4 = re.sub(r'\S*https?:\S*', '', t3)
    
    #remove #_something
    t5 = re.sub('#+', '', t4)
    
    #lower case each letter / case folding
    t6 = t5.lower()
    
    #alphabets only, exclude numbers and special characters
    t7 = re.sub(r'[^a-zA-Z]', ' ', t6)
    
    #Replace contractions with their extended forms
    t8 = contractions.fix(t7)
    return t8

In [150]:
file = input('Enter File Name:') #raw_tweet

df = pd.read_csv('raw_tweets_' + file + '.csv', usecols=['created_at', 'date', 'user_id', 'username', 'name', 'tweet', 'link'])
df

Enter File Name: psbb liputan6


,created_at,date,user_id,username,name,tweet,link
0,1.595920e+12,2020-07-28 14:03:33,47596019,liputan6dotcom,LIPUTAN6,"FOTO: PSBB Dilonggarkan, Pasar Apartemen Sewa ...",https://twitter.com/liputan6dotcom/status/1288...
1,1.595913e+12,2020-07-28 12:14:03,47596019,liputan6dotcom,LIPUTAN6,Belanja Pemerintah Terganggu Kebijakan PSBB h...,https://twitter.com/liputan6dotcom/status/1287...
2,1.595494e+12,2020-07-23 15:45:04,47596019,liputan6dotcom,LIPUTAN6,Survei Indikator Politik: Pengusaha dan Masyar...,https://twitter.com/liputan6dotcom/status/1286...
3,1.595294e+12,2020-07-21 08:15:34,47596019,liputan6dotcom,LIPUTAN6,Sejumlah Kota di Indonesia Kerja Sama dengan W...,https://twitter.com/liputan6dotcom/status/1285...
4,1.595247e+12,2020-07-20 19:05:05,47596019,liputan6dotcom,LIPUTAN6,"Relaksasi PSBB, Jumlah Pasien Suspect COVID-19...",https://twitter.com/liputan6dotcom/status/1285...
...,...,...,...,...,...,...,...
95,1.592378e+12,2020-06-17 14:10:03,47596019,liputan6dotcom,LIPUTAN6,PLN Diminta Beri Kompensasi Lonjakan Tagihan L...,https://twitter.com/liputan6dotcom/status/1273...
96,1.592367e+12,2020-06-17 11:11:02,47596019,liputan6dotcom,LIPUTAN6,100 Ribu Kendaraan Masuk Jakarta selama PSBB T...,https://twitter.com/liputan6dotcom/status/1273...
97,1.592355e+12,2020-06-17 07:42:33,47596019,liputan6dotcom,LIPUTAN6,Anies Sebut Tidak Ada Tanda Jakarta Kembali ke...,https://twitter.com/liputan6dotcom/status/1273...
98,1.592354e+12,2020-06-17 07:35:04,47596019,liputan6dotcom,LIPUTAN6,Deddy Mizwar dan PPFI Desak Penerbitan Protoko...,https://twitter.com/liputan6dotcom/status/1273...


In [151]:
for i, r in df.iterrows():
    y = cleansing(r['tweet'])
    df.loc[i, 'tweet'] = y
    
df.head()

,created_at,date,user_id,username,name,tweet,link
0,1.595920e+12,2020-07-28 14:03:33,47596019,liputan6dotcom,LIPUTAN6,foto psbb dilonggarkan pasar apartemen sewa ...,https://twitter.com/liputan6dotcom/status/1288...
1,1.595913e+12,2020-07-28 12:14:03,47596019,liputan6dotcom,LIPUTAN6,belanja pemerintah terganggu kebijakan psbb,https://twitter.com/liputan6dotcom/status/1287...
2,1.595494e+12,2020-07-23 15:45:04,47596019,liputan6dotcom,LIPUTAN6,survei indikator politik pengusaha dan masyar...,https://twitter.com/liputan6dotcom/status/1286...
3,1.595294e+12,2020-07-21 08:15:34,47596019,liputan6dotcom,LIPUTAN6,sejumlah kota di indonesia kerja sama dengan w...,https://twitter.com/liputan6dotcom/status/1285...
4,1.595247e+12,2020-07-20 19:05:05,47596019,liputan6dotcom,LIPUTAN6,relaksasi psbb jumlah pasien suspect covid ...,https://twitter.com/liputan6dotcom/status/1285...


In [152]:
#POS Tagger Dictionary
post_dict = {'J/':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}

In [153]:
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('indonesian')):
            newlist.append(tuple([word, post_dict.get(tag[0])]))
    return newlist

In [154]:
df['POS tagged'] = df['tweet'].apply(token_stop_pos)
df.head()

,created_at,date,user_id,username,name,tweet,link,POS tagged
0,1.595920e+12,2020-07-28 14:03:33,47596019,liputan6dotcom,LIPUTAN6,foto psbb dilonggarkan pasar apartemen sewa ...,https://twitter.com/liputan6dotcom/status/1288...,"[(foto, n), (psbb, n), (dilonggarkan, n), (pas..."
1,1.595913e+12,2020-07-28 12:14:03,47596019,liputan6dotcom,LIPUTAN6,belanja pemerintah terganggu kebijakan psbb,https://twitter.com/liputan6dotcom/status/1287...,"[(belanja, n), (pemerintah, n), (terganggu, n)..."
2,1.595494e+12,2020-07-23 15:45:04,47596019,liputan6dotcom,LIPUTAN6,survei indikator politik pengusaha dan masyar...,https://twitter.com/liputan6dotcom/status/1286...,"[(survei, n), (indikator, n), (politik, n), (p..."
3,1.595294e+12,2020-07-21 08:15:34,47596019,liputan6dotcom,LIPUTAN6,sejumlah kota di indonesia kerja sama dengan w...,https://twitter.com/liputan6dotcom/status/1285...,"[(kota, n), (indonesia, n), (kerja, n), (waze,..."
4,1.595247e+12,2020-07-20 19:05:05,47596019,liputan6dotcom,LIPUTAN6,relaksasi psbb jumlah pasien suspect covid ...,https://twitter.com/liputan6dotcom/status/1285...,"[(relaksasi, n), (psbb, n), (pasien, n), (susp..."


In [155]:
export = input('Enter File Name:') #cleaned_tweet

df.to_csv("cleaned_tweets_" + export + ".csv", index=False)

Enter File Name: psbb liputan6
